# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath
from azureml.data.dataset_factory import TabularDatasetFactory

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mobile_price_prediction'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [4]:
input_data = pd.read_csv("https://raw.githubusercontent.com/valeriapolozun/machine_learning_azure_capstone/803e04124532b8921a0b19d70aa71abaccf74a1f/Mobile_Price_Classification.csv")

In [ ]:
#input_data=pd.read_csv('Mobile_Price_Classification.csv')

In [5]:
input_data

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0.0,1,0,7,0.6,188.0,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1.0,0,1,53,0.7,136.0,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1.0,2,1,41,0.9,145.0,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0.0,0,0,10,0.8,131.0,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0.0,13,1,44,0.6,141.0,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,996,1700,1.0,1.9,0,0,1,54.0,0.5,170,...,17,644,913,2121,14,8,15,1,1,0
2996,997,609,0.0,1.8,1,0,0,13.0,0.9,186,...,2,1152,1632,1933,8,1,19,0,1,1
2997,998,1185,0.0,1.4,0,1,1,8.0,0.5,80,...,12,477,825,1223,5,0,14,1,0,0
2998,999,1533,1.0,0.5,1,0,0,50.0,0.4,171,...,12,38,832,2509,15,11,6,0,1,0


In [6]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(input_data,test_size=0.2)
train_data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
2244,245,1292,1.0,3.0,1,1,1,63.0,0.3,157,...,8,452,1309,1413,13,2,10,1,1,1
2039,40,567,0.0,2.6,0,0,0,2.0,0.2,87,...,1,181,566,2243,16,6,10,0,1,1
2867,868,1670,0.0,0.5,0,1,1,33.0,0.9,127,...,3,185,807,1830,5,0,6,1,1,1
2046,47,1094,0.0,2.3,1,2,0,20.0,0.9,135,...,5,221,610,1413,7,0,16,0,1,1
2245,246,1927,1.0,2.1,0,0,0,5.0,0.9,139,...,1,603,1228,2913,15,10,8,0,0,0


In [8]:
if "data" not in os.listdir():
    #os.mkdir("./training")if "training" not in os.listdir():
    os.mkdir("./data")

In [12]:
# Save training data to a csv file 
train_data.to_csv("data/mobile_price_traindata.csv", index=False)

In [13]:
data_store = ws.get_default_datastore()
data_store

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-b0f4c1c7-d8b2-4fdf-892e-1a0e9f0381bd",
  "account_name": "mlstrg142384",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [14]:
data_store.upload(src_dir='data/',target_path='pricing', overwrite=True, show_progress=True)


Uploading an estimated of 1 files
Uploading data/mobile_price_traindata.csv
Uploaded data/mobile_price_traindata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_0569ca4e90b04f05a07ca68a25b676ae

In [15]:
train_data = Dataset.Tabular.from_delimited_files(path = [(data_store, 'pricing/mobile_price_traindata.csv')])

In [16]:
train_data

{
  "source": [
    "('workspaceblobstore', 'pricing/mobile_price_traindata.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [22]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="price_range",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=5,
                             **automl_settings
)

In [18]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
mobile_price_prediction,AutoML_d7a83541-50a4-4b5f-b3fe-f441aa8e2a7d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
remote_run = experiment.submit(automl_config, show_output = True)

from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
mobile_price_prediction,AutoML_53d60ec2-9229-44ac-934f-289e492875b6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
mobile_price_prediction,AutoML_53d60ec2-9229-44ac-934f-289e492875b6,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_53d60ec2-9229-44ac-934f-289e492875b6',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-11T09:07:03.819819Z',
 'endTimeUtc': '2021-04-11T09:31:24.249983Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"mobile_price_prediction","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-142384","workspace_name":"quick-starts-ws-142384","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"v

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
##pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 25 kB/s /s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3
Note: you may need to restart the kernel to use updated packages.


In [24]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               objective='multi:softprob',
                                                                                               random_state=0,
                                                                                               reg_alpha=0,
                                                  

In [25]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [28]:
from azureml.core import Model

# Register model
best_automl.register_model(model_path='outputs/model.pkl', model_name='automl_model',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_automl.get_metrics()['accuracy']})



automl_model version: 1
	 Training context : Auto ML
	 Accuracy : 0.7941666666666667


hyperdrive_best_run.pkl version: 1




In [40]:
# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

automl_model version: 1
	 Training context : Auto ML
	 Accuracy : 0.7941666666666667




In [41]:
from azureml.automl.core.shared import constants
best_automl.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [42]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


env=Environment.from_conda_specification(name='myenv',file_path='myenv.yml')
inference_config = InferenceConfig(entry_script="scoring.py",environment=env)

In [43]:

from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)


In [ ]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="mobile-price-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)#deploy the model

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-11 10:23:22+00:00 Registering the environment.
2021-04-11 10:23:23+00:00 Use the existing image.
2021-04-11 10:23:23+00:00 Generating deployment configuration.
2021-04-11 10:23:24+00:00 Submitting deployment to compute.
2021-04-11 10:23:27+00:00 Checking the status of deployment mobile-price-deploy-service.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
import requests
import json

data={"data":
  [{'age': 87.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 981,
  'diabetes': 0,
  'ejection_fraction': 55,
  'high_blood_pressure': 1,
  'platelets': 368000,
  'serum_creatinine': 3.5,
  'serum_sodium': 137,
  'sex': 0,
  'smoking': 0,
  'time': 4}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())input_data=json.dumps(data)



TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:

service.delete()
compute_target.delete()
model.delete()